In [1]:
import requests
import pandas as pd
from datetime import datetime,date

In [2]:
def fetch_crypto_data(crypto_pair: str, start_date: str) -> pd.DataFrame:
    crypto_id, currency = crypto_pair.split('/')

    start_timestamp = int(datetime.strptime(start_date, '%Y-%m-%d').timestamp())

    url = f"https://api.coingecko.com/api/v3/coins/{crypto_id}/market_chart/range"

    params = {
        'vs_currency': currency.lower(),
        'from': start_timestamp,
        'to': int(datetime.now().timestamp()),  # Current time
    }

    response = requests.get(url, params=params)

    if response.status_code != 200:
        raise Exception(f"Error fetching data: {response.status_code} - {response.text}")

    data = response.json()

    if 'prices' not in data:
        raise Exception("No price data found in the response.")

    df = pd.DataFrame(data['prices'], columns=['timestamp', 'close'])

    df['Date'] = pd.to_datetime(df['timestamp'], unit='ms').dt.date
    
    df['Open'] = df['close']  # Placeholder, needs to be calculated
    df['High'] = df['close']  # Placeholder, needs to be calculated
    df['Low'] = df['close']   # Placeholder, needs to be calculated
    df['Close'] = df['close']

    # Group by date to calculate Open, High, Low, and Close
    df = df.groupby('Date').agg({
        'Open': 'first',
        'High': 'max',
        'Low': 'min',
        'Close': 'last'
    }).reset_index()

    return df

# Example usage
if __name__ == "__main__":
    crypto_pair = "bitcoin/usd"  # Use the CoinGecko ID for the cryptocurrency
    start_date = "2024-09-27"
    try:
        crypto_data = fetch_crypto_data(crypto_pair, start_date)
        print(crypto_data)
    except Exception as e:
        print(e)

          Date          Open          High           Low         Close
0   2024-09-26  65223.195668  65223.195668  64754.698699  65069.996931
1   2024-09-27  65130.768897  66304.167884  65001.916838  65884.023030
2   2024-09-28  65814.925133  66179.301394  65537.095386  65703.244989
3   2024-09-29  65869.088690  65955.127683  65483.576622  65586.452985
4   2024-09-30  65527.116446  65527.116446  63449.591154  63469.302673
5   2024-10-01  63356.222606  64080.501786  60773.960602  60791.996800
6   2024-10-02  60942.078966  61824.455066  60403.417353  60787.278563
7   2024-10-03  60627.058180  61350.308773  60047.210231  60888.677277
8   2024-10-04  60764.622319  62394.204091  60507.363739  62022.471620
9   2024-10-05  62094.413346  62307.202957  61764.447436  61991.285313
10  2024-10-06  61943.320592  62871.841848  61875.007999  62621.047500
11  2024-10-07  62811.799728  63794.841445  62301.390272  62301.390272
12  2024-10-08  62211.234201  62770.264257  61963.159443  62105.270602
13  20

In [3]:
today = date.today()
print(today)
if(today in crypto_data['Date'].values):
    today_data = crypto_data[crypto_data['Date']==today]
    print(today_data)
else:
    print("today does not exist")

2024-10-28
          Date          Open          High           Low         Close
32  2024-10-28  67942.828514  68968.777749  67615.685449  68568.197513


In [32]:
def calculate_crypto_metrics(df, variable1, variable2):
    # Rolling window calculations for high and low
    df['High_Last_{}_Days'.format(variable1)] = df['High'].rolling(window=variable1, min_periods=1).max()
    df['Low_Last_{}_Days'.format(variable1)] = df['Low'].rolling(window=variable1, min_periods=1).min()

    # Function to get the index of the last maximum or minimum
    def last_max_index(series):
        return series[::-1].idxmax()

    def last_min_index(series):
        return series[::-1].idxmin()

    # Calculate days since last high
    last_high_indices = df['High'].rolling(window=variable1, min_periods=1).apply(last_max_index, raw=False)
    df[f'Days_Since_High_Last_{variable1}_Days'] = (df.index - last_high_indices)

    # Calculate days since last low
    last_low_indices = df['Low'].rolling(window=variable1, min_periods=1).apply(last_min_index, raw=False)
    df[f'Days_Since_Low_Last_{variable1}_Days'] = (df.index - last_low_indices)
    

    #Percentage Difference Calculations
    df['%_Diff_From_High_Last_{}_Days'.format(variable1)] = (
        (df['Close'] - df['High_Last_{}_Days'.format(variable1)]) / df['High_Last_{}_Days'.format(variable1)] * 100
    )

    df['%_Diff_From_Low_Last_{}_Days'.format(variable1)] = (
        (df['Close'] - df['Low_Last_{}_Days'.format(variable1)]) / df['Low_Last_{}_Days'.format(variable1)] * 100
    )

    #Future High and Low Calculations
    df['High_Next_{}_Days'.format(variable2)] = df['High'].shift(-variable2).rolling(window=variable2, min_periods=1).max()
    df['Low_Next_{}_Days'.format(variable2)] = df['Low'].shift(-variable2).rolling(window=variable2, min_periods=1).min()

    #If Nan,then use the previous high value
    df['High_Next_{}_Days'.format(variable2)].fillna(method='ffill', inplace=True)
    df['Low_Next_{}_Days'.format(variable2)].fillna(method='ffill', inplace=True)
    
    # Percentage Difference from Future High and Low
    df['%_Diff_From_High_Next_{}_Days'.format(variable2)] = (
        (df['Close'] - df['High_Next_{}_Days'.format(variable2)]) / df['High_Next_{}_Days'.format(variable2)] * 100
    )

    df['%_Diff_From_Low_Next_{}_Days'.format(variable2)] = (
        (df['Close'] - df['Low_Next_{}_Days'.format(variable2)]) / df['Low_Next_{}_Days'.format(variable2)] * 100
    )
    
    return df

In [33]:
# Set your variable1 and variable2
variable1 = 7
variable2 = 5

# Call the function
df = calculate_crypto_metrics(crypto_data, variable1, variable2)

In [34]:
df

,Date,Open,High,Low,Close,High_Last_7_Days,Low_Last_7_Days,%_Diff_From_High_Last_7_Days,%_Diff_From_Low_Last_7_Days,High_Next_5_Days,Low_Next_5_Days,%_Diff_From_High_Next_5_Days,%_Diff_From_Low_Next_5_Days,Days_Since_High_Last_7_Days,Days_Since_Low_Last_7_Days
0,2024-09-26,65223.195668,65223.195668,64754.698699,65069.996931,65223.195668,64754.698699,-0.234884,0.486912,64080.501786,60773.960602,1.544144,7.068877,0.0,0.0
1,2024-09-27,65130.768897,66304.167884,65001.916838,65884.023030,66304.167884,64754.698699,-0.633663,1.744004,64080.501786,60403.417353,2.814462,9.073337,0.0,1.0
2,2024-09-28,65814.925133,66179.301394,65537.095386,65703.244989,66304.167884,64754.698699,-0.906312,1.464830,64080.501786,60047.210231,2.532351,9.419313,1.0,2.0
3,2024-09-29,65869.088690,65955.127683,65483.576622,65586.452985,66304.167884,64754.698699,-1.082458,1.284469,64080.501786,60047.210231,2.350093,9.224813,2.0,3.0
4,2024-09-30,65527.116446,65527.116446,63449.591154,63469.302673,66304.167884,63449.591154,-4.275546,0.031066,64080.501786,60047.210231,-0.953799,5.699003,3.0,0.0
5,2024-10-01,63356.222606,64080.501786,60773.960602,60791.996800,66304.167884,60773.960602,-8.313461,0.029678,62871.841848,60047.210231,-3.308071,1.240335,4.0,0.0
6,2024-10-02,60942.078966,61824.455066,60403.417353,60787.278563,66304.167884,60403.417353,-8.320577,0.635496,63794.841445,60047.210231,-4.714430,1.232477,5.0,0.0
7,2024-10-03,60627.058180,61350.308773,60047.210231,60888.677277,66304.167884,60047.210231,-8.167647,1.401342,63794.841445,60507.363739,-4.555485,0.630194,6.0,0.0
8,2024-10-04,60764.622319,62394.204091,60507.363739,62022.471620,66179.301394,60047.210231,-6.281163,3.289514,63794.841445,60486.877097,-2.778234,2.538723,6.0,1.0
9,2024-10-05,62094.413346,62307.202957,61764.447436,61991.285313,65955.127683,60047.210231,-6.009908,3.237578,63794.841445,59019.105152,-2.827119,5.035963,6.0,2.0


In [35]:
df.to_excel('crypto.xlsx',sheet_name='CryptoData',index=False)